In [1]:
# activate julia environment in a controlled way
julia_environment = "../central_control_PyJuMP/Julia_src/activate.jl"
include(julia_environment)

using XLSX
using CSV
using GLPK
using DataFrames
using JuMP
using Ipopt
using Gurobi
#using Plots
     

 Activating environment at `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Manifest.toml`
  Resolving package versions...
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Manifest.toml`
  Resolving package versions...
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Manifest.toml`
  Resolving package versions...
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Manifest.toml`
  Resolving package versions...
No Changes to `~/code/Horizonta

Activating the Julia virtual environment


No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
No Changes to `~/code/Horizontal_Proper/Example/CentJulEnv/Manifest.toml`


Status `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [336ed68f] CSV v0.8.2
  [9961bab8] Cbc v0.9.1
  [e2554f3b] Clp v0.9.1
  [a93c6f00] DataFrames v0.22.7
  [864edb3b] DataStructures v0.18.13
  [60bf3e95] GLPK v0.15.3
  [5c1252a2] GeometryBasics v0.3.13
  [2e9cd046] Gurobi v0.10.3
  [87dc4568] HiGHS v0.3.0
  [7073ff75] IJulia v1.23.3
  [b6b21f68] Ipopt v0.9.1
  [4076af6c] JuMP v0.22.3
  [b8f27783] MathOptInterface v0.10.9
  [fdba3010] MathProgBase v0.7.8
  [91a5bcdd] Plots v1.15.2
  [731186ca] RecursiveArrayTools v2.20.0
  [82193955] SCIP v0.10.0
  [2913bbd2] StatsBase v0.33.21
  [fdbf4ff8] XLSX v0.8.4
  [ddb6d928] YAML v0.4.8
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics


## Read Data

In [ ]:
shared_cand = DataFrame(XLSX.readtable("../Input_Data/CandLine.csv", "Taul1", header=true)) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("../Input_Data/CandLineInt.csv", "Taul1", header=true)) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("../Input_Data/SharedEline.csv", "Taul1", header=true)) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("../Input_Data/Tran.csv", "Taul1", header=true)) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("../Input_Data/Gen.csv" , "Taul1", header=true)) #Dataframe of generators
load =  DataFrame(XLSX.readtable("../Input_Data/Load.csv", "Taul1", header=true)) #Dataframe of loads
scen_prob = DataFrame(CSV.File("../Input_Data/Scenario_Probability.csv", header=true)) #Dataframe of scenario probabilities
zone_summary = DataFrame(CSV.File("../Input_Data/Zone_Summary.csv", header=true)) #Dataframe of region-number of nodes
##An example of the load dataframe can be found as follows

In [ ]:
Random.seed!(123)
for i in 1:size(load,1)
    load.P_load1[i] = load.P_load1[i] * rand(24)
    load.P_load2[i] = load.P_load2[i] * rand(24)
    load.P_load3[i] = load.P_load3[i] * rand(24)
    load.P_load4[i] = load.P_load4[i] * rand(24)
end

In [2]:
l(i,s) = load[load.zoneNum .== i, [1,2,s+2]] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i
scen_weight(s) = scen_prob.scen_weight[scen_prob.scenario .== s,:]

,Zone,Nodes_Total
,Int64,Int64
1,1,14
2,2,30
3,3,5


In [4]:
S = nrow(scen_prob)

4

In [5]:
function bin(i,N,S,Tmax)
    P = zeros(nrow(zone_summary),maximum(zone_summary.Nodes_Total),S,Tmax); 
    #P[i,1:N,:] .= 1
    for c in 1:nrow(shared_c(i))
        P[shared_c(i).nodeZone1[c],shared_c(i).tNodeID1[c],:,:] .= 1
        P[shared_c(i).nodeZone2[c],shared_c(i).tNodeID2[c],:,:] .= 1
    end
    for h in 1:nrow(shared_e(i))
        P[shared_e(i).nodeZone1[h],shared_e(i).tNodeID1[h],:,:] .= 1
        P[shared_e(i).nodeZone2[h],shared_e(i).tNodeID2[h],:,:] .= 1
    end
    return(P)
end

bin (generic function with 1 method)

In [6]:
shared = bin(1,14,S,Tmax).*bin(2,30,S,Tmax).*bin(3,5,S,Tmax)
not_shared = (1 .- shared)

3×30×4 Array{Float64,3}:
[:, :, 1] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0

[:, :, 2] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0

[:, :, 3] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0

[:, :, 4] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.

In [7]:
MC(1)

2-element Array{Float64,1}:
 34.30293932
 55.0

In [8]:
function zonal_app(i, N, S, Tmax, lambda, shared_line_decision_var, int_line_decision_var, angle_self, angle_1, angle_2, bin_self, bin_1, bin_2)
    
    Mod4 = Model(Ipopt.Optimizer)
    ###Change it to IPOPT
    
    
    @variable(Mod4,0 <= gen_var[1:N,1:S,1:Tmax]) # generation at each node
    @variable(Mod4,shared_cand_flow[1:nrow(shared_cand),1:S,1:Tmax])  #Power flowing into shared candidate lines 
    @variable(Mod4,int_cand_flow[1:nrow(int_c(i)), 1:S,1:Tmax]) #Power flowing into internal candidate lines 
    #@variable(Mod4,0 <= shared_cand_angle[1:nrow(shared_cand), 1:S, 1:2]<= 2*pi) #Phase angle decision for shared candidate lines
    #@variable(Mod4,0 <= int_cand_angle[1:nrow(int_c(i)), 1:S, 1:2]<= 2*pi) #Phase angle decision for internal candidate lines
    @variable(Mod4,shared_ex_flow[1:nrow(shared_ex), 1:S,1:Tmax])  #Power flowing on shared existing shared lines
    @variable(Mod4,int_ex_flow[1:nrow(int_e(i)), 1:S,1:Tmax])  #Power flowing on internal existing lines
    @variable(Mod4,0 <= node_voltage_phase_angle[1:nrow(zone_summary),1:maximum(zone_summary.Nodes_Total), 1:S,1:Tmax] <= 2*pi)
   # @variable(Mod4,0 <= shared_ex_angle[1:nrow(shared_ex), 1:S, 1:2]<= 2*pi) #Phase angle for existing shared lines
    #@variable(Mod4,0 <= int_ex_angle[1:nrow(int_e(i)), 1:S, 1:2]<= 2*pi) #Phase angle for internal existing lines
    
    omega = (55)^2 ;
    eta = 55;
    
    @expression(Mod4, total_cost, sum(scen_weight(s)[1].* sum((gen_var[n,s,t] .* sum((g(i).gNodeID .== n) .* MC(i))) for n in 1:N) for s in 1:S)
        .+ sum(sum(sum(eta * bin_self .* bin_1 .* (node_voltage_phase_angle .* (angle_self .- angle_1))
        .+ eta * bin_self .* bin_2 .* (node_voltage_phase_angle.* (angle_self .- angle_2))
        .+ (omega /2) * (bin_self .* (node_voltage_phase_angle.- angle_self)) .^ 2
        .+ (omega /2) * (bin_self .* bin_1 .* (node_voltage_phase_angle.- angle_1)) .^ 2
        .+ (omega /2) * (bin_self .* bin_2 .* (node_voltage_phase_angle .- angle_2)) .^ 2
        .+ node_voltage_phase_angle .* lambda))))
             
    for t in 1:Tmax        
        for s in 1:S        
                
            for n in 1:N
                # Power balance constraint for each node
                @constraint(Mod4, sum((g(i).gNodeID .== n) .* gen_var[n,s,t]) .+ sum(l(i,s)[:,3].* (l(i,s).lNodeID .== n))[t]./100 .==
                    sum((shared_cand.tNodeID1 .== n) .*bin_c(i) .* shared_cand_flow[:,s,t]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(i) .* shared_cand_flow[:,s,t]) .+
                    sum((shared_ex.tNodeID1 .== n) .* bin_e(i) .* shared_ex_flow[:,s,t]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(i) .* shared_ex_flow[:,s,t]) .+
                    sum((int_c(i).tNodeID1 .== n) .* int_cand_flow[:,s,t]) .- sum((int_c(i).tNodeID2 .== n) .* int_cand_flow[:,s,t]) .+
                    sum((int_e(i).tNodeID1 .== n) .* int_ex_flow[:,s,t]) .- sum((int_e(i).tNodeID2 .== n) .* int_ex_flow[:,s,t]))
                #Lower limit for generation of each node
                @constraint(Mod4, sum(g(i).gNodeID .== n) .* gen_var[n,s,t] .<= sum((g(i).gNodeID .== n) .* g(i).PgMax./100))
                #Upper limit for generation of each node
                @constraint(Mod4, sum((g(i).gNodeID .== n) .* g(i).PgMin./100) .<= sum(g(i).gNodeID .== n) .* gen_var[n,s,t])
            end

            #Constraints elated to the candidate shared lines:
            for c in 1:nrow(shared_cand) 
                #Associating the power flowing within the line with the phase angles of nodes:
                @constraint(Mod4, bin_c(i) .* shared_line_decision_var[c] .* shared_cand_flow[c,s,t] .== 
                    (1 ./ shared_cand.reacT[c]) .* (bin_c(i) .* 
                    (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s,t] 
                    .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s,t])))
                #limiting the upper bound of power flow flowing within candidate shared lines
                @constraint(Mod4, bin_c(i) .* shared_cand_flow[c,s,t] .<= bin_c(i) .* 
                    shared_line_decision_var[c] .* shared_cand.ptMax[c] ./ 100)
                #Limiting the lower bound of power flowing within the candidate shared lines
                @constraint(Mod4, -shared_cand.ptMax[c] .* bin_c(i) .* shared_line_decision_var[c] ./ 100 .<= bin_c(i) .* 
                    shared_cand_flow[c,s,t])
                #@constraint(Mod4, shared_cand_angle[5,2] .== 0)
            end

            #The same constraints related to the candidate internal lines:
            for c in 1:nrow(int_c(i))
                @constraint(Mod4,int_line_decision_var[c].*int_cand_flow[c,s,t] .== (1 ./ int_c(i).reacT[c]) .*
                        (node_voltage_phase_angle[int_c(i).zoneNum[c],int_c(i).tNodeID1[c],s,t] .-
                        node_voltage_phase_angle[int_c(i).zoneNum[c],int_c(i).tNodeID2[c],s,t]))
                @constraint(Mod4, int_cand_flow[c,s,t] .<= int_line_decision_var[c] .* int_c(i).ptMax[c] ./ 100)
                @constraint(Mod4, -int_c(i).ptMax[c] .*int_line_decision_var[c] ./ 100 .<= int_cand_flow[c,s,t])
            end

            #The same constraints related to the shared existing lines:
            for h in 1:nrow(shared_ex)
                @constraint(Mod4, bin_e(i) .* shared_ex_flow[h,s,t] .== (1 ./ shared_ex.reacT[h]) .*
                    (bin_e(i) .* (node_voltage_phase_angle[shared_ex.nodeZone1[h],shared_ex.tNodeID1[h],s,t] .-
                    node_voltage_phase_angle[shared_ex.nodeZone2[h],shared_ex.tNodeID2[h],s,t])))
                @constraint(Mod4, bin_e(i) .* shared_ex_flow[h,s,t] .<= bin_e(i) .* shared_ex. ptMax[h] ./ 100)
                @constraint(Mod4, bin_e(i) .* -shared_ex.ptMax[h] ./ 100 .<= bin_e(i) .* shared_ex_flow[h,s,t])
                #@constraint(Mod4, shared_ex_angle[1,2] .== 0)
                #@constraint(Mod4, shared_ex_angle[7,2] .== 0)

            end
            #The same constraints related to the internal existing lines:

            for h in 1:nrow(int_e(i))
                @constraint(Mod4, int_ex_flow[h,s,t] .== (1 ./ int_e(i).reacT[h]) 
                    .* (node_voltage_phase_angle[int_e(i).zoneNum[h],int_e(i).tNodeID1[h],s,t] .- 
                        node_voltage_phase_angle[int_e(i).zoneNum[h],int_e(i).tNodeID2[h],s,t]))
                @constraint(Mod4, int_ex_flow[h,s,t] .<= int_e(i).ptMax[h] ./ 100)
                @constraint(Mod4, -int_e(i).ptMax[h] ./ 100 .<= int_ex_flow[h,s,t])
            end
        end
        #The model objective is to minimize the total cost:
        
        @objective(Mod4, Min, total_cost)
        optimize!(Mod4)
        if termination_status(Mod4) == MOI.OPTIMAL #If the solution is optimal then it returns the variables
        
            shared_cand_power = value.(shared_cand_flow)
            shared_ex_power = value.(shared_ex_flow)
            phase_angle = value.(node_voltage_phase_angle)
            gen_power = value.(gen_var)
            int_cand_power = value.(int_cand_flow)
            obj_value = objective_value(Mod4)
            [obj_value,shared_cand_power,phase_angle,shared_ex_power,gen_power, int_cand_power]

        end
    end
end
        
    

zonal_app (generic function with 1 method)

In [9]:
f=ones(12000)
h=0
lambda_1= zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S,Tmax)
lambda_2= zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S,Tmax)
lambda_3 = zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S,Tmax)

angle_1 = zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S,Tmax)
angle_2 = zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S,Tmax)
angle_3 = zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S,Tmax)

cand_flow_1 =zeros(12000,nrow(shared_cand),S,Tmax)
cand_flow_2 =zeros(12000,nrow(shared_cand),S,Tmax)
cand_flow_3 =zeros(12000,nrow(shared_cand),S,Tmax)

ex_flow_1 = zeros(12000,nrow(shared_ex),S,Tmax)
ex_flow_2 = zeros(12000,nrow(shared_ex),S,Tmax)
ex_flow_3 = zeros(12000,nrow(shared_ex),S,Tmax)

int_cand_flow_1 = zeros(12000,nrow(int_c(1)),S,Tmax)
int_cand_flow_2 = zeros(12000,nrow(int_c(2)),S,Tmax)
int_cand_flow_3 = zeros(12000,nrow(int_c(3)),S,Tmax)

gen_1 = zeros(12000,zone_summary.Nodes_Total[1],S,Tmax)
gen_2 = zeros(12000,zone_summary.Nodes_Total[2],S,Tmax)
gen_3 = zeros(12000,zone_summary.Nodes_Total[3],S,Tmax)

iter = 0



shared_line_decision_var_m =  
[0.0
 0.0
 0.0
 1.0
 0.0
 0.0]
int_line_decision_var_1 =
[0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0]
int_line_decision_var_2 =
[0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0]
int_line_decision_var_3 =
[0.0
 0.00]

2-element Array{Float64,1}:
 0.0
 0.0

In [10]:
    j = 2
    max_iter = 5000
    for j in 2:max_iter    # It considers 1000 iteration
        if abs(f[j-1]) > 0.0005  
         
            R_1 = zonal_app(1, 14, S, lambda_1[j,:,:,:], shared_line_decision_var_m, int_line_decision_var_1,
            angle_1[j-1,:,:,:], angle_2[j-1,:,:,:], angle_3[j-1,:,:,:], bin(1,14,S,Tmax), bin(2,30,S,Tmax), bin(3,5,S,Tmax))
        
            R_2 = zonal_app(2, 30, S, lambda_2[j,:,:,:], shared_line_decision_var_m, int_line_decision_var_2,
            angle_2[j-1,:,:,:], angle_1[j-1,:,:,:], angle_3[j-1,:,:,:], bin(2,30,S,Tmax), bin(1,14,S,Tmax), bin(3,5,S,Tmax))
        
            R_3 = zonal_app(3, 5, S, lambda_3[j,:,:,:], shared_line_decision_var_m, int_line_decision_var_3,
            angle_3[j-1,:,:,:], angle_1[j-1,:,:,:], angle_2[j-1,:,:,:], bin(3,5,S,Tmax), bin(1,14,S,Tmax), bin(2,30,S,Tmax))
        
            
        
        
            lambda_1[j+1,:,:,:] .= bin(1,14,S,Tmax) .* lambda_1[j,:,:,:] 
                .+ 0.0001 * bin(1,14,S,Tmax) .* not_shared .* (R_1[3] .- (bin(2,30,S,Tmax) .* R_2[3] .+ bin(3,5,S,Tmax) .* R_3[3]))
                .+ 0.0001 * bin(1,14,S,Tmax) .* shared .* (R_1[3] .- (bin(2,30,S,Tmax) .* R_2[3] .+ bin(3,5,S,Tmax) .* R_3[3]) ./ 2)
            lambda_2[j+1,:,:,:] .= bin(2,30,S) .* lambda_2[j,:,:,:] 
                .+ 0.0001 * bin(2,30,S,Tmax) .* not_shared .* (R_2[3] .- (bin(1,14,S,Tmax) .* R_1[3] .+ bin(3,5,S,Tmax) .* R_3[3]))
                .+ 0.0001 * bin(2,30,S,Tmax) .* shared .* (R_2[3] .- (bin(1,14,S,Tmax) .* R_1[3] .+ bin(3,5,S,Tmax) .* R_3[3]) ./ 2)
            lambda_3[j+1,:,:,:] .= bin(3,5,S) .* lambda_3[j,:,:,:] 
                .+ 0.0001 * bin(3,5,S,Tmax).* not_shared .* (R_3[3] .- (bin(1,14,S,Tmax) .* R_1[3] .+ bin(2,30,S,Tmax) .* R_2[3]))
                .+ 0.0001 * bin(3,5,S,Tmax).* shared .* (R_3[3] .- (bin(1,14,S,Tmax) .* R_1[3] .+ bin(2,30,S,Tmax) .* R_2[3]) ./ 2)
        
        
        
            angle_1[j,:,:,:] = R_1[3]
            angle_2[j,:,:,:] = R_2[3]
            angle_3[j,:,:,:] = R_3[3]
            
            cand_flow_1[j,:,:] = R_1[2] .* 100
            cand_flow_2[j,:,:] = R_2[2] .* 100
            cand_flow_3[j,:,:] = R_3[2] .* 100
        
            ex_flow_1[j,:,:,:] = R_1[4] .* 100
            ex_flow_2[j,:,:,:] = R_2[4] .* 100
            ex_flow_3[j,:,:,:] = R_3[4] .* 100
        
            gen_1[j,:,:] = R_1[5] .* 100
            gen_2[j,:,:] = R_2[5] .* 100
            gen_3[j,:,:] = R_3[5] .* 100
        
            
            f[j] = sum(sum(bin(1,14,S,Tmax) .* not_shared .* (R_1[3] .- (bin(2,30,S,Tmax) .* R_2[3] .+ bin(3,5,S,Tmax) .* R_3[3])) .^ 2)) +
                    sum(sum(bin(2,30,S,Tmax) .* not_shared .* (R_2[3] .- (bin(1,14,S,Tmax) .* R_1[3] .+ bin(3,5,S,Tmax) .* R_3[3])) .^ 2)) +
                    sum(sum(bin(3,5,S,Tmax).* not_shared .* (R_3[3] .- (bin(1,14,S,Tmax) .* R_1[3] .+ bin(2,30,S,Tmax) .* R_2[3])) .^ 2)) +
                    sum(sum(bin(1,14,S,Tmax) .* shared .* (R_1[3] .- (bin(2,30,S,Tmax) .* R_2[3] .+ bin(3,5,S,Tmax) .* R_3[3])./ 2) .^ 2)) +
                    sum(sum(bin(2,30,S,Tmax) .* shared .* (R_2[3] .- (bin(1,14,S,Tmax) .* R_1[3] .+ bin(3,5,S,Tmax) .* R_3[3])./ 2) .^ 2)) +
                    sum(sum(bin(3,5,S,Tmax).* shared .* (R_3[3] .- (bin(1,14,S,Tmax) .* R_1[3] .+ bin(2,30,S,Tmax) .* R_2[3])./ 2) .^ 2))

            h= R_2[1] + R_1[1] + R_3[1]
            
            if isdefined(Main, :IJulia)
                Main.IJulia.stdio_bytes[] = 0
            end
        
            print('*') 
        
        else
            iter = j-1
            print(iter)
            break
        end
    end
    


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1312
Number of nonzeros in inequality constraint Jacobian.:      768
Number of nonzeros in Lagrangian Hessian.............:       52

Total number of variables............................:      592
                     variables with only lower bounds:       56
                variables with lower and upper bounds:      360
                     variables with only upper bounds:        0
Total number of equal

LoadError: MethodError: no method matching getindex(::Nothing, ::Int64)

In [ ]:
if iter == 0
    iter = max_iter
end

In [ ]:
iter

In [ ]:
h #OF of the second stage

In [ ]:
f[iter]

In [ ]:
using Plots
plot(2:iter,f[2:iter])

In [ ]:
bin(1,14,S) .* bin(3,5,S) .* (angle_1[iter,:,:,:]-angle_3[iter,:,:,:])

In [ ]:
angle_1[iter,:,:,:]

In [ ]:
angle_2[iter,:,:,:]

In [ ]:
angle_3[iter,:,:,:]

In [ ]:
bin_c(1)

In [ ]:
bin_c(2)

In [ ]:
bin_c(3)

In [ ]:
cand_flow_1[iter,:,:]

In [ ]:
cand_flow_2[iter,:,:]

In [ ]:
cand_flow_3[iter,:,:]

In [ ]:
ex_flow_1[iter,:,:]

In [ ]:
ex_flow_2[iter,:,:]

In [ ]:
ex_flow_3[iter,:,:]

In [ ]:
gen_1[iter,:,:]

In [ ]:
gen_2[iter,:,:]

In [ ]:
gen_3[iter,:,:]

In [ ]:
gen_2[iter,:,:]